In [15]:
import json
import os
import pandas
import psycopg2
import random
import requests
import spotipy
import time
import uuid
from spotipy.oauth2 import SpotifyOAuth

In [2]:
# read environment variables from local config file

config_directory = os.getcwd() + '/../local/'
exec(open(config_directory+'config.py').read())

In [73]:
scope = "playlist-read-private"
spotify_connection = spotipy.Spotify(auth_manager=SpotifyOAuth(scope=scope))

In [194]:
class MusicLibrary:

    def __init__(self):
        
        self.data = {
            # objects
            'album': {},
            'artist': {},
            'genre': {},
            'label': {},
            'playlist': {},
            'track': {},
            # maps
            'album_genre': {},
            'album_label': {},
            'album_track': {},
            'artist_genre': {},
            'playlist_track': {},
            'track_artist': {},
            # utility
            'playlist_backlog': {},
        }
        self.words = []
        self.spotify_connection = spotipy.Spotify(auth_manager=SpotifyOAuth(scope="playlist-read-private"))
        self.db_connection = psycopg2.connect(
            dbname=os.environ['LOCAL_DB_DBNAME'],
            user=os.environ['LOCAL_DB_USER'],
            password=os.environ['LOCAL_DB_PASSWORD'],
            host=os.environ['LOCAL_DB_HOST'], 
            port=os.environ['LOCAL_DB_PORT'],
        )
        self.cursor = self.db_connection.cursor()


    def create_uuid(self, **kwargs):
        input_string = ''
        namespace = uuid.NAMESPACE_DNS
        for key, value in kwargs.items():
            input_string += str(key) + ':' + str(value) + ';'
        return str(uuid.uuid5(namespace, input_string))


    def add(self, object_name, key, value):
        success = False
        try:
            if object_name in self.data:
                self.data[object_name][key] = value
                success = True
            else:
                print('Error: named object does not exist')
        except Exception as e:
            print(e)
        return success
        

    def get(self, object_name, key=None):
        obj = None
        if object_name in self.data:
            obj = self.data[object_name]
        else:
            print('Error: named object does not exist')
        if key is None:
            return obj
        elif key in obj:
            return obj[key]
        else:
            return None


    def delete(self, object_name, key):
        success = False
        try:
            if object_name in self.data:
                del self.data[object_name][key]
                success = True
            else:
                print('Error: named object does not exist')
        except Exception as e:
            print(e)
        return success


    def check(self, object_name, key, instance=None):
        success = False
        lib_instance = self.get(object_name, key)
        if instance is not None and lib_instance is not None:
            if lib_instance == instance:
                success = True
        elif instance is None and lib_instance is not None:
            success = True
        return success


    def fetch(self, object_name, key):
        obj = None
        try:
            if object_name == 'album':
                obj = self.spotify_connection.album(key)
            elif object_name == 'artist':
                obj = self.spotify_connection.artist(key)
            elif object_name == 'playlist':
                obj = self.spotify_connection.playlist(key)
            elif object_name == 'track':
                obj = self.spotify_connection.track(key)
            time.sleep(1)
        except Exception as e:
            print(e)
            time.sleep(2)
        return obj


    def write(self, **kwargs):
        success = False
        temp_data = dict(self.data)
        try:
            for obj_name, obj_data in kwargs['metadata'].items():
                for key, value in obj_data.items():
                    if self.check(obj_name, key) is True:
                        self.delete(obj_name, key)
                    self.add(obj_name, key, value)
            success = True
        except Exception as e:
            self.data = temp_data
            print(e)
        return success


    def process_genre(self, key, **kwargs):

        # evaluate existing metadata object
        if 'metadata' in kwargs:
            metadata = kwargs['metadata']
        else:
            metadata = {}
            
        # evaluate existing object type
        if 'genre' not in metadata:
            metadata['genre'] = {}

        # add the passed value
        try:
            metadata['genre'][key] = kwargs['value']
        except Exception as e:
            metadata = None
            print(e)

        return metadata


    def process_label(self, key, **kwargs):

        # evaluate existing metadata object
        if 'metadata' in kwargs:
            metadata = kwargs['metadata']
        else:
            metadata = {}
            
        # evaluate existing object type
        if 'label' not in metadata:
            metadata['label'] = {}

        # add the passed value
        try:
            metadata['label'][key] = kwargs['value']
        except Exception as e:
            metadata = None
            print(e)

        return metadata


    def process_map(self, map_name, key, value, **kwargs):

        # evaluate existing metadata object
        if 'metadata' in kwargs:
            metadata = kwargs['metadata']
        else:
            metadata = {}
            
        # evaluate existing object type
        if map_name not in metadata:
            metadata[map_name] = {}

        # add the passed value
        metadata[map_name][key] = value

        return metadata


    def process_album(self, key, **kwargs):

        # evaluate existing metadata object
        if 'metadata' in kwargs:
            metadata = kwargs['metadata']
        else:
            metadata = {}

        # evaluate existing object type
        if 'album' not in metadata:
            metadata['album'] = {}

        # translate track data to object
        if 'value' in kwargs:
            value = kwargs['value']
        else:
            value = self.fetch('album', key)
            if value is None:
                return None
        album_object = {
            'name': value['name'],
            'release_date': value['release_date'],
            'url': value['external_urls']['spotify'],
            'total_tracks': value['total_tracks'],
            'popularity': value['popularity'],
        }
            
        # fetch album associated objects to process
        genre_ids = []
        album_genre_ids = []
        for genre in value['genres']:
            genre_id = self.create_uuid(genre=genre)
            genre_ids.append(genre_id)
            ag_id = self.create_uuid(album=key, genre=genre_id)
            album_genre_ids.append(ag_id)
        label_id = self.create_uuid(label=value['label'])
        album_label_id = self.create_uuid(album=key, label=label_id)

        # test if album data already exists in library
        add_data = False
        if self.check('album', key, album_object) is True:
            if self.check('label', label_id) is False:
                add_data = True
            if self.check('album_label', album_label_id) is False:
                add_data = True
            for genre_id in genre_ids:
                if add_data is True:
                    break
                if self.check('genre', genre_id) is False:
                    add_data = True
            for album_genre_id in album_genre_ids:
                if add_data is True:
                    break
                if self.check('album_genre', album_genre_id) is False:
                    add_data = True
        else:
            add_data = True

        # conditionally append metadata for writing
        if add_data is True:
            metadata['album'][key] = album_object
            al_value = {
                'album_id': key,
                'label_id': label_id,
            }
            metadata = self.process_map('album_label', album_label_id, al_value, metadata=metadata)
            metadata = self.process_label(label_id, metadata=metadata, value={'name': value['label']})
            for genre in value['genres']:
                if metadata is None:
                    break  
                genre_id = self.create_uuid(genre=genre)
                ag_id = self.create_uuid(album=key, genre=genre_id)
                ag_value = {
                    'album_id': key,
                    'genre_id': genre_id,
                }
                metadata = self.process_map('album_genre', ag_id, ag_value, metadata=metadata)
                metadata = self.process_genre(genre_id, metadata=metadata, value={'name': genre})

        return metadata


    def process_artist(self, key, **kwargs):

        # evaluate existing metadata object
        if 'metadata' in kwargs:
            metadata = kwargs['metadata']
        else:
            metadata = {}

        # evaluate existing object type
        if 'artist' not in metadata:
            metadata['artist'] = {}

        # translate track data to object
        if 'value' in kwargs:
            value = kwargs['value']
        else:
            value = self.fetch('artist', key)
            if value is None:
                return None
        artist_object = {
            'name': value['name'],
            'url': value['external_urls']['spotify'],
            'followers': value['followers']['total'],
            'popularity': value['popularity'],
        }
            
        # fetch artist associated objects to process
        genre_ids = []
        artist_genre_ids = []
        for genre in value['genres']:
            genre_id = self.create_uuid(genre=genre)
            genre_ids.append(genre_id)
            ag_id = self.create_uuid(artist=key, genre=genre_id)
            artist_genre_ids.append(ag_id)

        # test if album data already exists in library
        add_data = False
        if self.check('artist', key, artist_object) is True:
            for genre_id in genre_ids:
                if add_data is True:
                    break
                if self.check('genre', genre_id) is False:
                    add_data = True
            for artist_genre_id in artist_genre_ids:
                if add_data is True:
                    break
                if self.check('artist_genre', artist_genre_id) is False:
                    add_data = True
        else:
            add_data = True

        # conditionally append metadata for writing
        if add_data is True:
            metadata['artist'][key] = artist_object
            for genre in value['genres']:
                if metadata is None:
                    break  
                genre_id = self.create_uuid(genre=genre)
                ag_id = self.create_uuid(artist=key, genre=genre_id)
                ag_value = {
                    'artist_id': key,
                    'genre_id': genre_id,
                }
                metadata = self.process_map('artist_genre', ag_id, ag_value, metadata=metadata)
                metadata = self.process_genre(genre_id, metadata=metadata, value={'name': genre})

        return metadata


    def process_track(self, key, **kwargs):

        # evaluate existing metadata object
        if 'metadata' in kwargs:
            metadata = kwargs['metadata']
        else:
            metadata = {}

        # evaluate existing object type
        if 'track' not in metadata:
            metadata['track'] = {}

        # translate track data to object
        if 'value' in kwargs:
            value = kwargs['value']
        else:
            value = self.fetch('track', key)
            if value is None:
                return None
        track_object = {
            'name': value['name'],
            'duration_ms': value['duration_ms'],
            'popularity': value['popularity'],
            'url': value['external_urls']['spotify'],
            'isrc': value['external_ids']['isrc'],
        }
            
        # fetch track associated objects to process
        artist_ids = []
        track_artist_ids = []
        for artist in value['artists']:
            artist_ids.append(artist['id'])
            ta_id = self.create_uuid(track=key, artist=artist['id'])
            track_artist_ids.append(ta_id)
        album_id = value['album']['id']
        album_track_id = self.create_uuid(album=value['album']['id'], track=key)

        # test if track data already exists in library
        add_data = False
        if self.check('track', key, track_object) is True:
            if self.check('album', album_id) is False:
                add_data = True
            if self.check('album_track', album_track_id) is False:
                add_data = True
            for artist_id in artist_ids:
                if add_data is True:
                    break
                if self.check('artist', artist_id) is False:
                    add_data = True
            for track_artist_id in track_artist_ids:
                if add_data is True:
                    break
                if self.check('track_artist', track_artist_id) is False:
                    add_data = True
        else:
            add_data = True

        # conditionally append metadata for writing
        if add_data is True:
            metadata['track'][key] = track_object
            at_value = {
                'album_id': value['album']['id'],
                'track_id': key,
                'track_number': value['track_number'],
            }
            metadata = self.process_map('album_track', album_track_id, at_value, metadata=metadata)
            metadata = self.process_album(value['album']['id'], metadata=metadata)
            for artist_data in value['artists']:
                if metadata is None:
                    break  
                ta_id = self.create_uuid(track=key, artist=artist_data['id'])
                ta_value = {
                    'track_id': key,
                    'artist_id': artist_data['id'],
                    'type': artist_data['type'],
                }
                metadata = self.process_map('track_artist', ta_id, ta_value, metadata=metadata)
                metadata = self.process_artist(artist_data['id'], metadata=metadata)

        return metadata


    def process_playlist(self, key, **kwargs):

        # evaluate existing metadata object
        if 'metadata' in kwargs:
            metadata = kwargs['metadata']
        else:
            metadata = {}

        # evaluate existing object type
        if 'playlist' not in metadata:
            metadata['playlist'] = {}

        # translate playlist data to object
        if 'value' in kwargs:
            value = kwargs['value']
        else:
            value = self.fetch('playlist', key)
            if value is None:
                return None
        playlist_object = {
            'name': value['name'],
            'description': value['description'],
            'url': value['external_urls']['spotify'],
            'total_tracks': value['tracks']['total'],
        }
            
        # fetch playlist tracks to process
        try:
            pt_fetch = self.spotify_connection.playlist_tracks(key, limit=value['tracks']['total'])
            time.sleep(1)
            track_ids = []
            playlist_track_ids = []
            for track_data in pt_fetch['items']:
                track_ids.append(track_data['track']['id'])
                pt_id = self.create_uuid(playlist=key, track=track_data['track']['id'])
                playlist_track_ids.append(pt_id)
        except Exception as e:
            print(e)
            time.sleep(2)
            return None

        # test if playlist data already exists in library
        add_data = False
        if self.check('playlist', key, playlist_object) is True:
            for track_id in track_ids:
                if add_data is True:
                    break
                if self.check('track', track_id) is False:
                    add_data = True
            for playlist_track_id in playlist_track_ids:
                if add_data is True:
                    break
                if self.check('playlist_track', playlist_track_id) is False:
                    add_data = True
        else:
            add_data = True

        # conditionally append metadata for writing
        if add_data is True:
            metadata['playlist'][key] = playlist_object
            for track_data in pt_fetch['items']:
                pt_id = self.create_uuid(playlist=key, track=track_data['track']['id'])
                pt_value = {
                    'playlist_id': key,
                    'track_id': track_data['track']['id'],
                    'added_at': track_data['added_at'],
                }
                metadata = self.process_map('playlist_track', pt_id, pt_value, metadata=metadata)
                metadata = self.process_track(track_data['track']['id'], metadata=metadata)
                if metadata is None:
                    break                
            
        return metadata

    
    def load_playlist_backlog(self, limit=20):
        success = False

        # protect against invalid limits
        if limit > 50:
            limit = 50
            print('Limit truncated to 50.  Do not attempt a higher limit!')
        elif limit < 1:
            limit = 1
            print('Limit truncated to 1.  Do not attempt a lower limit!')

        try:
            # load data into local memory
            temp_backlog = {}
            n = 0
            c = limit
            while c == limit:
                results = self.spotify_connection.current_user_playlists(limit=limit, offset=n*limit)
                time.sleep(1)
                i = 0
                for idx, item in enumerate(results['items']):
                    temp_backlog[item['id']] = item
                    i += 1
                n += 1
                c = i

            # process local data into class object
            for key, value in temp_backlog.items():
                if key in self.data['playlist_backlog']:
                    del self.data['playlist_backlog'][key]
                self.data['playlist_backlog'][key] = value
            success = True
                
        except Exception as e:
            print(e)
            time.sleep(2)

        return success


    def process_a_backlogged_playlist(self):
        success = False

        # choose the next playlist
        key = next(iter(self.data['playlist_backlog']))
        value = self.data['playlist_backlog'][key]

        # process the playlist into metadata
        metadata = self.process_playlist(key, value=value)

        # conditionally write the metadata and clean up
        if metadata is not None:
            self.write(metadata=metadata)
            del self.data['playlist_backlog'][key]
            success = True
        elif metadata is None and self.check('playlist', key) is True:
            del self.data['playlist_backlog'][key]

        return success


    def get_object_dataframe(self, object_name):
        objects = self.get(object_name)
        object_array = []
        for key, item in objects.items():
            object_instance = {'id': key}
            for field, value in item.items():
                object_instance[field] = value
            object_array.append(object_instance)
        return pandas.DataFrame.from_records(object_array)


    def write_object_data_to_db(self, object_name):
        success = False
        df = self.get_object_dataframe(object_name)
        
        try:
            for index, row in df.iterrows():
                self.cursor.execute("""SELECT * FROM """ + object_name + """ WHERE id = %s;""", (str(row['id']), ))
                results = self.cursor.fetchall()
                if len(results) == 0:
                    c = ''
                    v = ''
                    for col in df.columns:
                        if row[col]:
                            if v != '':
                                v += ', '
                                c += ', '
                            c += col
                            if type(row[col]) == str:
                                if 'date' in col:
                                    # still needs to debug year only edge case
                                    v += "'" + row[col] + "'::DATE"
                                else:
                                    v += "'" + row[col].replace("'","") + "'"
                            else:
                                v += str(row[col])
                    command = """INSERT INTO """ + object_name + """ (""" + c + """) VALUES (""" + v + """);"""
                    self.cursor.execute(command)
                    self.db_connection.commit()
                    print('Command sent: ' + command)
            success = True
                    
        except Exception as e:
            print(e)

        return success

    
    def extract_words(self):
        success = False
    
        # words from existing music library
        if len(self.data['track']) >= 1:
            for key, value in self.data['track'].items():
                for word in value['name'].split():
                    if word.lower() not in self.words:
                        self.words.append(word.lower())
                        success = True
            
        # random english words from a site
        # else:
        #     response = requests.get("https://www.mit.edu/~ecprice/wordlist.10000")
        #     byte_words = response.content.splitlines()
        #     for bw in byte_words:
        #         words.append(bw.decode('utf-8'))

        return success

    
    def get_word_query(self):
        if len(self.words) >= 1:
            return f'%{random.choice(self.words)}%'
        else:
            return 'house'


    def get_random_track(self, query=None):
        if query is None:
            query = self.get_word_query()
        try:
            pre_search = spotify_connection.search(q=query, type='track', market='US')
            time.sleep(1)
            random_offset = random.randrange(pre_search['tracks']['total'])
            results = spotify_connection.search(q=query, type='track', offset=random_offset, market='US')
            time.sleep(1)
            item = results['tracks']['items'][0]
            return item
        except Exception as e:
            print(e)
            time.sleep(2)
            return None

In [170]:
print(lib.extract_words())
query = lib.get_word_query()
print(lib.words)
print(query)
print()

track = lib.get_random_track(query)
print(track['name'])
data = lib.process_track(track['id'], value=track)
print(lib.write(metadata=data))
print()

print(lib.extract_words())
print(lib.words)
print()

['house', 'of', 'the', "risin'", 'sun', 'every', 'breath', 'you', 'take', 'it', 'off']
%of%

Better Off Alone
True

True
['house', 'of', 'the', "risin'", 'sun', 'every', 'breath', 'you', 'take', 'it', 'off', 'jolie', 'fille', 'i', 'adore', '(feat.', 'daecolm)', 'too', 'cool', 'to', 'be', 'careless', 'move', 'never', 'walk', 'alone', 'believe', 'we', 'watch', 'my', 'love', '(2024)', 'no', 'bad', 'vibes', 'u', 'ready', 'acid', 'carnival', 'other', 'side', 'techno', 'cosmic', 'ride', '-', 'rebrand', 'mix', 'play', 'extended', 'running', 'high', 'jiggler', 'remix', 'fresh', 'beat', 'connect', 'avalon', 'touchtalk', 'luana', 'olivier', 'giacomotto', 'desert', 'original', 'wolf', 'kiko', 'lazy', 'moet', 'primal', 'source', 'arcanum', 'rakara', 'dancing', 'in', 'night', 'naiw', 'interdimensional', 'triangle', 'secret', 'playground', 'better']



In [169]:
# lib = MusicLibrary()
# print(lib.load_playlist_backlog(50))

print()
print(len(lib.data['playlist']))
print(len(lib.data['playlist_backlog']))

looper = 0
while looper < 2:
    if lib.process_a_backlogged_playlist() is True:
        looper += 1


print()
print(len(lib.data['playlist']))
print(len(lib.data['playlist_backlog']))


0
155

2
153


In [176]:
# dump to JSON if needed

filepath = os.path.join(config_directory, "spotipy_data.json")
with open(filepath, 'w') as f:
    json.dump(lib.data, f, indent=4)

In [174]:
album_df = lib.get_object_dataframe('album')
album_df.info()
print()
print(album_df.head())
print()
print('id max length: ' + str(album_df['id'].astype(str).str.len().max()))
print('name max length: ' + str(album_df['name'].astype(str).str.len().max()))
print('release_date example: ' + str(album_df['release_date'][0]))
print('url max length: ' + str(album_df['url'].astype(str).str.len().max()))
print('total_tracks max value: ' + str(album_df['total_tracks'].max()))
print('popularity max value: ' + str(album_df['popularity'].max()))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33 entries, 0 to 32
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            33 non-null     object
 1   name          33 non-null     object
 2   release_date  33 non-null     object
 3   url           33 non-null     object
 4   total_tracks  33 non-null     int64 
 5   popularity    33 non-null     int64 
dtypes: int64(2), object(4)
memory usage: 1.7+ KB

                       id                             name release_date  \
0  5k63xxy9YcKM0H9GS3vP1K                        Bob Dylan   1962-03-19   
1  5W9OT0a5iZlBr83a9WMKFY  Synchronicity (Remastered 2003)   1983-06-17   
2  6fpLLJsDSSAlToEDW2jv4F        Animal (Expanded Edition)   2010-01-01   
3  7x1lI7V6hPlNkQtcuRCrnV                      Jolie Fille   2024-07-26   
4  5MrdEMJrBnauBJsd1nYY00      I Adore You (feat. Daecolm)   2024-07-19   

                                                 url  total_

In [175]:
lib.cursor.execute("""
    CREATE TABLE IF NOT EXISTS album (
        id VARCHAR(25),
        name VARCHAR(100),
        release_date DATE,
        url VARCHAR(80),
        total_tracks INTEGER,
        popularity INTEGER
    );
""")
lib.db_connection.commit()

In [188]:
print(lib.write_object_data_to_db('album'))

Added Album: 5k63xxy9YcKM0H9GS3vP1K
Added Album: 5W9OT0a5iZlBr83a9WMKFY
Added Album: 6fpLLJsDSSAlToEDW2jv4F
Added Album: 7x1lI7V6hPlNkQtcuRCrnV
Added Album: 5MrdEMJrBnauBJsd1nYY00
Added Album: 17zk2lz0xesn8Y4YncohZa
Added Album: 4M8ThNHbNjda2cPwL8XZHS
Added Album: 5B6cE705BTRvviiO1p4m0o
Added Album: 6fsPFECJJpDQQpbh1mYtwu
Added Album: 73PdhRzMikkkImZ4qHOzZ3
Added Album: 0fniUtJNAYz6eSOE9HJe32
Added Album: 1AegI4Bn648fulynzE7YML
Added Album: 0qn64q9MVIlHoUnfdcsSN9
Added Album: 0k9Gu6Gw0Xu4RSzaKaaFZi
Added Album: 7aMQPXovqv7PGp1YCcVptp
Added Album: 1NsMu7kOA3PbMi43wQBiJD
Added Album: 3nn7SebGCHkTjcMVRSwUAR
Added Album: 7e6gi422iHsPTmlPPYaWqd
Added Album: 79f4YGIHehV62UyKk2g2ew
Added Album: 6GNjxWwhOKiUuyC008Nnwi
Added Album: 3XlfXLQaS599e1Ps2ji6i6
Added Album: 3579IFF1Qkp6EnfFWgxsma
Added Album: 5sqnyWzxYqsFiKGqlnTAZh
Added Album: 1IBxvfyn7h1QEzy0Fv6M82
Added Album: 18nc4yYWxmddVfrDAkli7C
Added Album: 435PDbc4Bcl7goTCYuJMMr
Added Album: 7dYr1ps1bX959ga8c26WpV
Added Album: 6nVw63KRpVrfHuu

In [191]:
artist_df = lib.get_object_dataframe('artist')
artist_df.info()
print()
print(artist_df.head())
print()
print('id max length: ' + str(artist_df['id'].astype(str).str.len().max()))
print('name max length: ' + str(artist_df['name'].astype(str).str.len().max()))
print('url max length: ' + str(artist_df['url'].astype(str).str.len().max()))
print('followers max value: ' + str(artist_df['followers'].max()))
print('popularity max value: ' + str(artist_df['popularity'].max()))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55 entries, 0 to 54
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          55 non-null     object
 1   name        55 non-null     object
 2   url         55 non-null     object
 3   followers   55 non-null     int64 
 4   popularity  55 non-null     int64 
dtypes: int64(2), object(3)
memory usage: 2.3+ KB

                       id        name  \
0  74ASZWbe4lXaubB36ztrGX   Bob Dylan   
1  5NGO30tJxFlKixkPSgXcFE  The Police   
2  6LqNN22kT3074XbTVUrhzX       Kesha   
3  6gYwbDKcqhLitCTlgF1oZn         Maz   
4  3of9b8iUxyXtfnwwahfUNx      Antdot   

                                                 url  followers  popularity  
0  https://open.spotify.com/artist/74ASZWbe4lXaub...    7116056          77  
1  https://open.spotify.com/artist/5NGO30tJxFlKix...    7111896          79  
2  https://open.spotify.com/artist/6LqNN22kT3074X...    8696724          82  
3  https

In [173]:
lib.cursor.execute("""
    CREATE TABLE IF NOT EXISTS artist (
        id VARCHAR(25),
        name VARCHAR(100),
        url VARCHAR(80),
        followers INTEGER,
        popularity INTEGER
    );
""")
lib.db_connection.commit()

In [192]:
for index, row in artist_df.iterrows():
    lib.cursor.execute("""SELECT * FROM artist WHERE id = %s;""", (str(row['id']), ))
    results = lib.cursor.fetchall()
    if len(results) == 0:
        c = ''
        v = ''
        for col in artist_df.columns:
            if row[col]:
                if v != '':
                    v += ', '
                    c += ', '
                c += col
                if type(row[col]) == str:
                    v += "'" + row[col].replace("'","") + "'"
                else:
                    v += str(row[col])
        lib.cursor.execute("""INSERT INTO artist (""" + c + """) VALUES (""" + v + """);""")
        lib.db_connection.commit()
        print("Added Artist: " + row['id'])

Added Artist: 6gYwbDKcqhLitCTlgF1oZn
Added Artist: 3of9b8iUxyXtfnwwahfUNx
Added Artist: 4Fx9JQTrRogacwiF4Drhoy
Added Artist: 7MkQpNkd549FwPBbgkGyE8
Added Artist: 5PlfkPxwCpRRWQJBxCa0By
Added Artist: 0u6GtibW46tFX7koQ6uNJZ
Added Artist: 7hQmAXAzWI6D350VTgkKTG
Added Artist: 1IFAU4mznUcfPVP9z2c24N
Added Artist: 4E0HD2PMY8kQJIjlShrLUS
Added Artist: 2loEsOijJ6XiGzWYFXMIRk
Added Artist: 6XjgqvaVLUib6lCerbZrYD
Added Artist: 26WKgv73kRHD0gEDKD1i8j
Added Artist: 2yyaNPcdxZcn4iutCcLMaC
Added Artist: 37lTKHi901VnwngQ7C8noT
Added Artist: 6zsJjoCtL1WByG0VsuFWzR
Added Artist: 5qMHOzLlXeOEjOncWYtRfZ
Added Artist: 1iZiG82D4w7FLHvOUUj4zW
Added Artist: 7cR62TCmcorAdyny40NsP0
Added Artist: 6PTNNcLg90Kkl89JcEwKhT
Added Artist: 1dgdvbogmctybPrGEcnYf6
Added Artist: 4ScCswdRlyA23odg9thgIO
Added Artist: 7zAAwgV5Wqmvpb4GzvlRkP
Added Artist: 4QGD0m9AGZixhuPAzaBeD7
Added Artist: 34L6CF5Bc3ogSvcGfq9TnL
Added Artist: 5NafK2XHLrFYXioRE8ElPQ
Added Artist: 3UCPEgnaf1ZoEkrGOXFxhV
Added Artist: 2uC37Er4dfALwAHVh0rOMj
A

In [193]:
genre_df = lib.get_object_dataframe('genre')
genre_df.info()
print()
print(genre_df.head())
print()
print('id max length: ' + str(genre_df['id'].astype(str).str.len().max()))
print('name max length: ' + str(genre_df['name'].astype(str).str.len().max()))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26 entries, 0 to 25
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      26 non-null     object
 1   name    26 non-null     object
dtypes: object(2)
memory usage: 548.0+ bytes

                                     id               name
0  8826c15b-bca8-5450-8480-a8dc979a1353          folk rock
1  e6f2f4ee-c1fc-594a-a307-6c1f08af2635               folk
2  543638c0-0b94-54ce-b6d2-6963ea1abcaa  singer-songwriter
3  cd60813f-31a2-535d-987a-a29879f8acb8         roots rock
4  295395a7-3a26-596c-9368-879cc61209f5       country rock

id max length: 36
name max length: 17


In [181]:
lib.cursor.execute("""
    CREATE TABLE IF NOT EXISTS genre (
        id VARCHAR(40),
        name VARCHAR(40)
    );
""")
lib.db_connection.commit()

In [ ]:
for index, row in genre_df.iterrows():
    lib.cursor.execute("""SELECT * FROM genre WHERE id = %s;""", (str(row['id']), ))
    results = lib.cursor.fetchall()
    if len(results) == 0:
        c = ''
        v = ''
        for col in artist_df.columns:
            if row[col]:
                if v != '':
                    v += ', '
                    c += ', '
                c += col
                if type(row[col]) == str:
                    v += "'" + row[col].replace("'","") + "'"
                else:
                    v += str(row[col])
        lib.cursor.execute("""INSERT INTO artist (""" + c + """) VALUES (""" + v + """);""")
        lib.db_connection.commit()
        print("Added Artist: " + row['id'])

In [182]:
label_df = lib.get_object_dataframe('label')
label_df.info()
print()
print(label_df.head())
print()
print('id max length: ' + str(label_df['id'].astype(str).str.len().max()))
print('name max length: ' + str(label_df['name'].astype(str).str.len().max()))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28 entries, 0 to 27
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      28 non-null     object
 1   name    28 non-null     object
dtypes: object(2)
memory usage: 580.0+ bytes

                                     id                 name
0  43643583-d562-552c-9e4e-bec87450c3b6             Columbia
1  a396f5b4-3faf-5bf1-89e8-faaf6b624e91      Polydor Records
2  dc0851fb-a4b9-510a-8159-b94205476388    RCA Records Label
3  ce0c2692-b624-582d-986d-7d71c3f1c086  Dawn Patrol Records
4  5d198c2a-945a-5b3f-b612-0103d23456e7               Virgin

id max length: 36
name max length: 63


In [183]:
lib.cursor.execute("""
    CREATE TABLE IF NOT EXISTS label (
        id VARCHAR(40),
        name VARCHAR(80)
    );
""")
lib.db_connection.commit()

In [185]:
# playlist_df = lib.get_object_dataframe('playlist')
playlist_df.info()
print()
print(playlist_df.head())
print()
print('id max length: ' + str(playlist_df['id'].astype(str).str.len().max()))
print('name max length: ' + str(playlist_df['name'].astype(str).str.len().max()))
print('description max length: ' + str(playlist_df['description'].astype(str).str.len().max()))
print('url max length: ' + str(playlist_df['url'].astype(str).str.len().max()))
print('total_tracks max value: ' + str(playlist_df['total_tracks'].max()))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            2 non-null      object
 1   name          2 non-null      object
 2   description   2 non-null      object
 3   url           2 non-null      object
 4   total_tracks  2 non-null      int64 
dtypes: int64(1), object(4)
memory usage: 212.0+ bytes

                       id    name description  \
0  7205ZsM0CqmcCvily2Q4h7   Ça Va               
1  2Eh4kUPPV20JBqaWjZv4lt  Bunker               

                                                 url  total_tracks  
0  https://open.spotify.com/playlist/7205ZsM0Cqmc...             9  
1  https://open.spotify.com/playlist/2Eh4kUPPV20J...            21  

id max length: 22
name max length: 6
description max length: 0
url max length: 56
total_tracks max value: 21


In [186]:
lib.cursor.execute("""
    CREATE TABLE IF NOT EXISTS playlist (
        id VARCHAR(25),
        name VARCHAR(100),
        description VARCHAR(200),
        url VARCHAR(80),
        total_tracks INTEGER
    );
""")
lib.db_connection.commit()

In [177]:
track_df = lib.get_object_dataframe('track')
track_df.info()
print()
print(track_df.head())
print()
print('id max length: ' + str(track_df['id'].astype(str).str.len().max()))
print('name max length: ' + str(track_df['name'].astype(str).str.len().max()))
print('duration_ms max value: ' + str(track_df['duration_ms'].max()))
print('popularity max value: ' + str(track_df['popularity'].max()))
print('url max length: ' + str(track_df['url'].astype(str).str.len().max()))
print('isrc max length: ' + str(track_df['isrc'].astype(str).str.len().max()))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34 entries, 0 to 33
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           34 non-null     object
 1   name         34 non-null     object
 2   duration_ms  34 non-null     int64 
 3   popularity   34 non-null     int64 
 4   url          34 non-null     object
 5   isrc         34 non-null     object
dtypes: int64(2), object(4)
memory usage: 1.7+ KB

                       id                         name  duration_ms  \
0  3yFRbFaSj8MeNuYaN21HIu      House of the Risin' Sun       318013   
1  1JSTJqkT5qHq8MDJnJbRE1        Every Breath You Take       253920   
2  0WCiI0ddWiu5F2kSHgfw5S                  Take It Off       215200   
3  3pyPyRSFvIOMNS3S2CeMDB                  Jolie Fille       194442   
4  5nPbKG04fhLkIAjcPFaZq7  I Adore You (feat. Daecolm)       214000   

   popularity                                                url          isrc  
0          

In [178]:
lib.cursor.execute("""
    CREATE TABLE IF NOT EXISTS track (
        id VARCHAR(25),
        name VARCHAR(100),
        duration_ms INTEGER,
        popularity INTEGER,
        url VARCHAR(80),
        isrc VARCHAR(20)
    );
""")
lib.db_connection.commit()

In [ ]:
# close database connection

connection.close()

In [91]:
#results = spotify_connection.current_user_playlists(limit=50)
another_playlist = spotify_connection.playlist('7205ZsM0CqmcCvily2Q4h7')
raw_playlist_data = results['items'][0]
for key, value in another_playlist.items():
    print(str(key) + ': ' + str(value))
    print()

collaborative: False

description: 

external_urls: {'spotify': 'https://open.spotify.com/playlist/7205ZsM0CqmcCvily2Q4h7'}

followers: {'href': None, 'total': 0}

href: https://api.spotify.com/v1/playlists/7205ZsM0CqmcCvily2Q4h7?additional_types=track

id: 7205ZsM0CqmcCvily2Q4h7

images: [{'height': 640, 'url': 'https://mosaic.scdn.co/640/ab67616d00001e0205005dc483e9f8a73ccc6d3dab67616d00001e022ae2ed5bc30ffd941d29d4acab67616d00001e024843e0c207ca63169070fb74ab67616d00001e02a959d221820e650bee1ca487', 'width': 640}, {'height': 300, 'url': 'https://mosaic.scdn.co/300/ab67616d00001e0205005dc483e9f8a73ccc6d3dab67616d00001e022ae2ed5bc30ffd941d29d4acab67616d00001e024843e0c207ca63169070fb74ab67616d00001e02a959d221820e650bee1ca487', 'width': 300}, {'height': 60, 'url': 'https://mosaic.scdn.co/60/ab67616d00001e0205005dc483e9f8a73ccc6d3dab67616d00001e022ae2ed5bc30ffd941d29d4acab67616d00001e024843e0c207ca63169070fb74ab67616d00001e02a959d221820e650bee1ca487', 'width': 60}]

name: Ça Va

owner: {'di

In [90]:
pt_fetch = spotify_connection.playlist_tracks(raw_playlist_data['id'], limit=raw_playlist_data['tracks']['total'])
raw_track_data = pt_fetch['items'][0]
for key, value in raw_track_data.items():
    print(str(key) + ': ' + str(value))
    print()

added_at: 2025-04-27T20:33:07Z

added_by: {'external_urls': {'spotify': 'https://open.spotify.com/user/12154158077'}, 'href': 'https://api.spotify.com/v1/users/12154158077', 'id': '12154158077', 'type': 'user', 'uri': 'spotify:user:12154158077'}

is_local: False

primary_color: None

track: {'preview_url': None, 'available_markets': ['AG', 'AR', 'BB', 'BO', 'BR', 'BS', 'BZ', 'CA', 'CL', 'CO', 'CR', 'DM', 'DO', 'EC', 'GD', 'GT', 'GY', 'HN', 'HT', 'JM', 'KN', 'LC', 'MX', 'NI', 'PA', 'PE', 'PR', 'PY', 'SR', 'SV', 'TT', 'US', 'UY', 'VC', 'VE'], 'explicit': False, 'type': 'track', 'episode': False, 'track': True, 'album': {'available_markets': ['AG', 'AR', 'BB', 'BO', 'BR', 'BS', 'BZ', 'CA', 'CL', 'CO', 'CR', 'DM', 'DO', 'EC', 'GD', 'GT', 'GY', 'HN', 'HT', 'JM', 'KN', 'LC', 'MX', 'NI', 'PA', 'PE', 'PR', 'PY', 'SR', 'SV', 'TT', 'US', 'UY', 'VC', 'VE'], 'type': 'album', 'album_type': 'single', 'href': 'https://api.spotify.com/v1/albums/7x1lI7V6hPlNkQtcuRCrnV', 'id': '7x1lI7V6hPlNkQtcuRCrnV', 